In [2]:
import pandas as pd

# IMDB

In [2]:
imdb0 = pd.read_csv("./training-curves/results.csv")
imdb1 = pd.read_csv("./training-curves/results (1).csv") 
imdb2 = pd.read_csv("./training-curves/results (2).csv")
imdb3 = pd.read_csv("./training-curves/results (3).csv")
imdb4 = pd.read_csv("./training-curves/imdb0.csv")
imdb5 = pd.read_csv("./training-curves/imdb1.csv")
imdb6 = pd.read_csv("./training-curves/imdb2.csv")
imdb7 = pd.read_csv("./training-curves/imdb3.csv")

In [6]:
imdb0.head()

,Trial-ID,Status,Iteration,batch_size,dropout,gpu,hidden_dim,lr,max_features,recurrent_dropout,Objective,acc,loss,val_loss
0,1,INTERMEDIATE,0,128,0.630026,1,176,0.001662,20000,0.310548,0.79752,0.73108,0.535280,0.445803
1,3,INTERMEDIATE,0,128,0.630026,1,176,0.001662,20000,0.310548,0.76684,0.73092,0.532583,0.479070
2,2,INTERMEDIATE,0,128,0.630026,1,176,0.001662,20000,0.310548,0.78636,0.73116,0.534804,0.457844
3,1,INTERMEDIATE,1,128,0.630026,1,176,0.001662,20000,0.310548,0.83864,0.83608,0.381884,0.383688
4,3,INTERMEDIATE,1,128,0.630026,1,176,0.001662,20000,0.310548,0.84116,0.83688,0.377870,0.365818


In [7]:
imdb1.head()

,Trial-ID,Status,Iteration,batch_size,dropout,gpu,hidden_dim,lr,max_features,recurrent_dropout,Objective,acc,loss,val_loss
0,3,INTERMEDIATE,0,128,0.185486,2,198,0.004934,20000,0.242921,0.80156,0.69760,0.574149,0.428400
1,2,INTERMEDIATE,0,128,0.185486,2,198,0.004934,20000,0.242921,0.65612,0.71100,0.560769,0.865297
2,1,INTERMEDIATE,0,128,0.185486,2,198,0.004934,20000,0.242921,0.72676,0.69312,0.569137,0.576050
3,3,INTERMEDIATE,1,128,0.185486,2,198,0.004934,20000,0.242921,0.85024,0.86480,0.318536,0.339944
4,2,INTERMEDIATE,1,128,0.185486,2,198,0.004934,20000,0.242921,0.84464,0.86124,0.328870,0.348360


In [21]:
imdb1.loc[:, 'Trial-ID'] = imdb1.loc[:, 'Trial-ID'] + imdb0['Trial-ID'].max()
imdb2.loc[:, 'Trial-ID'] = imdb2.loc[:, 'Trial-ID'] + imdb1['Trial-ID'].max()
imdb3.loc[:, 'Trial-ID'] = imdb3.loc[:, 'Trial-ID'] + imdb2['Trial-ID'].max()
imdb4.loc[:, 'Trial-ID'] = imdb4.loc[:, 'Trial-ID'] + imdb3['Trial-ID'].max()
imdb5.loc[:, 'Trial-ID'] = imdb5.loc[:, 'Trial-ID'] + imdb4['Trial-ID'].max()
imdb6.loc[:, 'Trial-ID'] = imdb6.loc[:, 'Trial-ID'] + imdb5['Trial-ID'].max()
imdb7.loc[:, 'Trial-ID'] = imdb7.loc[:, 'Trial-ID'] + imdb6['Trial-ID'].max()

In [22]:
imdbnew = pd.concat([imdb0, imdb1, imdb2, imdb3, imdb4, imdb5, imdb6, imdb7])

In [26]:
imdbnew.to_csv("training-curves/imdb_complete.csv", index=False)

# IMDB CuDNN

In [3]:
cudnnimdb = [pd.read_csv("./imdblstm/imdb_cudnn_{}.csv".format(i)) for i in range(1,6)]

In [4]:
for i in range(1,5):
    cudnnimdb[i].loc[:, 'Trial-ID'] = cudnnimdb[i].loc[:, 'Trial-ID'] + cudnnimdb[i-1]['Trial-ID'].max()

In [5]:
cudnnimdb_merged = pd.concat(cudnnimdb)

In [6]:
cudnnimdb_merged['Trial-ID'].max()

16209

In [7]:
cudnnimdb_completed = cudnnimdb_merged.query("Status == 'COMPLETED'").sort_values(by="Trial-ID").drop(['Status', 'Iteration', 'test_acc'], axis=1)

In [8]:
(cudnnimdb_completed.groupby('dropout_embedding').agg('count').loc[:,'Trial-ID'] != 25).sum()

97

In [9]:
len(cudnnimdb_completed.groupby('dropout_embedding').agg('count').loc[:,'Trial-ID'])

651

In [10]:
cudnnimdb_final = pd.merge(cudnnimdb_completed, cudnnimdb_merged.loc[cudnnimdb_merged.test_acc.notnull(),['Trial-ID', 'test_acc']], on=['Trial-ID'] )

In [11]:
cudnnimdb_final.head()

,Objective,Trial-ID,acc,batch_size,decay,dropout_embedding,dropout_lstm,embedding_regularizer,hidden_dim,kernel_regularizer,loss,lr,max_features,recurrent_regularizer,rho,val_loss,test_acc
0,0.8016,1,0.79570,16,3.318116e-08,0.40268,0.09012,4.542953e-09,199,0.120845,0.480063,0.001411,38897,0.0371,0.870638,0.455114,0.80508
1,0.8272,2,0.84190,16,3.318116e-08,0.40268,0.09012,4.542953e-09,199,0.120845,0.407657,0.001411,38897,0.0371,0.870638,0.431528,0.83320
2,0.8276,3,0.84095,16,3.318116e-08,0.40268,0.09012,4.542953e-09,199,0.120845,0.408343,0.001411,38897,0.0371,0.870638,0.426137,0.83412
3,0.8276,4,0.84295,16,3.318116e-08,0.40268,0.09012,4.542953e-09,199,0.120845,0.407736,0.001411,38897,0.0371,0.870638,0.431300,0.83392
4,0.8280,5,0.84005,16,3.318116e-08,0.40268,0.09012,4.542953e-09,199,0.120845,0.408263,0.001411,38897,0.0371,0.870638,0.426169,0.83412


In [12]:
cudnnimdb_final.to_csv("imdblstm/imdb_results.csv", index=False)

# IMDB CuDNN Shuffle

In [28]:
cudnnimdb = [pd.read_csv("./imdblstm/imdb-shuffle-{}.csv".format(i)) for i in range(0,6)]

In [29]:
for i in range(1,6):
    cudnnimdb[i].loc[:, 'Trial-ID'] = cudnnimdb[i].loc[:, 'Trial-ID'] + cudnnimdb[i-1]['Trial-ID'].max()

3


In [30]:
cudnnimdb_merged = pd.concat(cudnnimdb)

In [31]:
cudnnimdb_merged['Trial-ID'].max()

13470

In [50]:
cudnnimdb_completed = cudnnimdb_merged.query("Status == 'COMPLETED'").sort_values(by="Trial-ID").drop(['Status', 'Iteration'], axis=1)

In [53]:
cudnnimdb_completed.to_csv("imdblstm/imdb_results_shuffled.csv", index=False)

In [58]:
(cudnnimdb_completed.groupby('dropout_embedding').agg('count').loc[:,'Trial-ID'] != 25).sum()

6

In [59]:
len(cudnnimdb_completed.groupby('dropout_embedding').agg('count').loc[:,'Trial-ID'])

539

# MNIST

In [13]:
mnist0 = pd.read_csv("./mnistcnn/results0.csv")
mnist1 = pd.read_csv("./mnistcnn/results1.csv")
mnist2 = pd.read_csv("./mnistcnn/results2.csv")
mnist3 = pd.read_csv("./mnistcnn/results3.csv")
mnist4 = pd.read_csv("./mnistcnn/results4.csv")

In [14]:
mnist0.head()

,Trial-ID,Status,Iteration,dropout,lr,lr_decay,top_dropout,Objective,test_loss
0,9,INTERMEDIATE,15,0.098702,0.042292,2.376943e-08,0.131907,0.044594,0.039489
1,3,INTERMEDIATE,15,0.098702,0.042292,2.376943e-08,0.131907,0.043856,0.040495
2,6,INTERMEDIATE,15,0.098702,0.042292,2.376943e-08,0.131907,0.048140,0.039094
3,9,COMPLETED,15,0.098702,0.042292,2.376943e-08,0.131907,0.044594,0.039489
4,6,COMPLETED,15,0.098702,0.042292,2.376943e-08,0.131907,0.048140,0.039094


In [6]:
mnist1.head()

,Trial-ID,Status,Iteration,dropout,lr,lr_decay,top_dropout,Objective,test_loss
0,1,INTERMEDIATE,15,0.455228,0.020329,0.000011,0.307316,0.036981,0.031062
1,7,INTERMEDIATE,15,0.455228,0.020329,0.000011,0.307316,0.042388,0.033308
2,4,INTERMEDIATE,15,0.455228,0.020329,0.000011,0.307316,0.037627,0.031520
3,9,INTERMEDIATE,15,0.455228,0.020329,0.000011,0.307316,0.038914,0.033915
4,3,INTERMEDIATE,15,0.455228,0.020329,0.000011,0.307316,0.037050,0.033154


In [7]:
mnist2.head()

,Trial-ID,Status,Iteration,dropout,lr,lr_decay,top_dropout,Objective,test_loss
0,4,INTERMEDIATE,15,0.049809,0.007612,0.000011,0.424734,0.077693,0.062142
1,6,INTERMEDIATE,15,0.049809,0.007612,0.000011,0.424734,0.069594,0.062726
2,2,INTERMEDIATE,15,0.049809,0.007612,0.000011,0.424734,0.060178,0.055121
3,3,INTERMEDIATE,15,0.049809,0.007612,0.000011,0.424734,0.059416,0.059143
4,5,INTERMEDIATE,15,0.049809,0.007612,0.000011,0.424734,0.056417,0.051852


In [8]:
mnist3.head()

,Trial-ID,Status,Iteration,dropout,lr,lr_decay,top_dropout,Objective,test_loss
0,9,INTERMEDIATE,15,0.464728,0.002626,2.951939e-07,0.472051,0.114417,0.100752
1,3,INTERMEDIATE,15,0.464728,0.002626,2.951939e-07,0.472051,0.126462,0.115474
2,6,INTERMEDIATE,15,0.464728,0.002626,2.951939e-07,0.472051,0.123477,0.111836
3,9,COMPLETED,15,0.464728,0.002626,2.951939e-07,0.472051,0.114417,0.100752
4,6,COMPLETED,15,0.464728,0.002626,2.951939e-07,0.472051,0.123477,0.111836


In [15]:
mnist1.loc[:, 'Trial-ID'] = mnist1.loc[:, 'Trial-ID'] + mnist0['Trial-ID'].max()
mnist2.loc[:, 'Trial-ID'] = mnist2.loc[:, 'Trial-ID'] + mnist1['Trial-ID'].max()
mnist3.loc[:, 'Trial-ID'] = mnist3.loc[:, 'Trial-ID'] + mnist2['Trial-ID'].max()
mnist4.loc[:, 'Trial-ID'] = mnist4.loc[:, 'Trial-ID'] + mnist3['Trial-ID'].max()

In [16]:
mnistnew = pd.concat([mnist0, mnist1, mnist2, mnist3, mnist4])

In [17]:
mnistnew.query("Status == 'COMPLETED'").to_csv("./mnistcnn/mnist_with_test_set_merged.csv", index=False)

# MNIST CNN Unshuffled

In [36]:
mnist_unshuffled = [pd.read_csv("./mnistcnn/mnistcnn-unshuffled-{}.csv".format(i)) for i in range(6)]

In [37]:
for i in range(1,6):
    mnist_unshuffled[i].loc[:, 'Trial-ID'] = mnist_unshuffled[i].loc[:, 'Trial-ID'] + mnist_unshuffled[i-1]['Trial-ID'].max()

In [38]:
mnist_unshuffled_merged = pd.concat(mnist_unshuffled)

In [39]:
mnist_unshuffled_merged['Trial-ID'].max()

15000

In [40]:
mnist_unshuffled_merged.query("Status == 'COMPLETED'").to_csv("./mnistcnn/mnist-unshuffled-merged.csv", index=False)

# Boston

In [16]:
n=4
name = 'boston-shuffled-'

In [17]:
boston = [pd.read_csv("./bostongbrt/{}{}.csv".format(name, i)) for i in range(n)]

In [18]:
for i in range(1,n):
    boston[i].loc[:, 'Trial-ID'] = boston[i].loc[:, 'Trial-ID'] + boston[i-1]['Trial-ID'].max()

In [19]:
boston_merged = pd.concat(boston)

In [20]:
boston_merged['Trial-ID'].max()

25000

In [21]:
boston_merged.query("Status == 'COMPLETED'").to_csv("./bostongbrt/{}merged.csv".format(name), index=False)